In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
import numpy.ma as ma
import os
import time

import pandas as pd
# from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import mixture, metrics

# from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob

# from radio_beam.commonbeam import getMinVolEllipse
#
# from ..src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
# from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset
#
# import plotly.graph_objects as go

from matplotlib.image import NonUniformImage
import matplotlib

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

from tqdm.notebook import tqdm
import json

from sklearn.metrics import pairwise_distances
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
import scipy
from sklearn import linear_model


## Распределение диаметров

In [ ]:
# pixel = 49.7 / 1000
pixel_1500_1500 = 50 / 983


# images = np.load('data/saved_metadata/images_4x_preprocess.npy')
images = np.load('data/generated_data/images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('artif.npy')
# images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('data/saved_np/images_10_mixer_preprocess.npy')

paths = np.load('data/generated_data/metadata_left_(1500,1500,1)_preprocessed.npy')
# paths = np.load('data/saved_np/metadata_10_mixer_preprocess.npy')
paths = [path.replace('\\','/') for path in paths]



# name = 'artif'
name = 'left_(1500,1500,1)'
# name = '10_mixer_preprocess'
# name = '4x_left_(1500,1500,1)'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

# types_dict = {'class_1': 'Type None'}

for step in range(1, 2):
    # print('step: ',step)

    grainGenerate.diametr_approx_save(save_path=name, images=images,paths=paths, types_dict=types_dict, step=step, pixel=pixel_1500_1500,max_images_num_per_class=1)

In [ ]:
def diametr_approx_all(json_data_path,saved_image_name, step, N, M, indices=None,save=False):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    data = open(json_data_path,encoding='utf-8')
    data = json.load(data)

    alloys_indices=range(len(data))

    if indices is not None:
        alloys_indices=indices

    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(N, M))
    fig.patch.set_facecolor('white')

    # ax1.set_ylabel('ln( p(x) )')
    # ax2.set_ylabel('ln( p(x) )')

    # ax2.set_xlabel('Большая полуось, мкм')
    # ax1.set_xlabel('Малая полуось, мкм', )

    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors = ['orange', 'red','blue','green', 'indigo']

    # linear approx
    for i in alloys_indices:

        ax1.plot(data['data'][i]['linear_approx_plot'][0]['a_beams'][0],
                 data['data'][i]['linear_approx_plot'][0]['a_beams'][1],color=colors[i])
        ax2.plot(data['data'][i]['linear_approx_plot'][0]['b_beams'][0],
                 data['data'][i]['linear_approx_plot'][0]['b_beams'][1],color=colors[i])

    legend_a=[]

    s=60
    # s=10

    # density curve
    for i in alloys_indices:
        marker = markers[i]
        ax1.scatter(data['data'][i]['density_curve_scatter'][0]['a_beams'][0],data['data'][i]['density_curve_scatter'][0]['a_beams'][1],  marker=marker,color=colors[i],s=s)
        ax2.scatter(data['data'][i]['density_curve_scatter'][0]['b_beams'][0],data['data'][i]['density_curve_scatter'][0]['b_beams'][1], marker=marker,color=colors[i],s=s)

        legend_a.append(data['data'][i]['name']+' '+data['data'][i]['type'])

    ax1.set_facecolor("White")
    ax2.set_facecolor("White")
    # ax1.legend(legend_a, loc=1)
    # ax2.legend(legend_a, loc=1)

    k=2
    plot_names=['a_beams','b_beams']
    # plt.yticks(np.arange(0,0.035,0.01))
    ax2.set_xticks(np.arange(0,12,3))
    if save:
        fig.subplots_adjust(wspace=0.25)
        plt.savefig(f'bimodal_beams_{saved_image_name}_step_{step}.png',bbox_inches='tight')
        img_plot=io.imread(f'bimodal_beams_{saved_image_name}_step_{step}.png')

        for i in range(k):
            step_img=img_plot.shape[1]/k
            io.imsave(f'bimodal_beams_{plot_names[i]}_{saved_image_name}_step_{step}.png',img_plot[:,int(i*step_img):int((i+1)*step_img)])

    plt.show()


In [ ]:
N = 15
M = 7

step = 2
save=True
# save=False


plt.rcParams['font.size'] = '30'

path=f'left_(1500,1500,1)_step_{step}_beams.json'
saved_image_name='original'


diametr_approx_all(path,saved_image_name, step, N, M, indices=[3,0,2],save=save)
# diametr_approx_all(path,saved_image_name, step, N, M, indices=[0, 2, 3],save=save)

## Heatmap для двумерного распределения полуосей

In [ ]:
folder = 'data/generated_data//beams'
name = 'left_(1500,1500,1)'
step=2
all_a_beams = np.load(f'{folder}/beams_original_values_a_step_left_(1500,1500,1)2.npy', allow_pickle=True)
all_b_beams = np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1)2.npy', allow_pickle=True)

In [ ]:
h_step=1
i=1
pixel = 49.7 / 1000
a_b_zeros=np.zeros((int((max(all_a_beams[i])*pixel))//h_step+1,int((max(all_b_beams[i])*pixel)//h_step+1)))
for j in range(len(all_a_beams[i])):

    key1=all_a_beams[i][j]
    key2=all_b_beams[i][j]

    a_b_zeros[all_a_beams[i][j]//h_step,all_b_beams[i][j]//h_step]+=1

x_sticks=np.arange(a_b_zeros.shape[0]*2,2)
y_sticks=np.arange(a_b_zeros.shape[1]*2,2)

# fig, ax = plt.subplots(figsize=(15,15))
# im = ax.imshow(a_b_zeros)
#
# # ax.set_xticks(np.arange(len(x_sticks)),
# #               x_sticks)
# # ax.set_yticks(np.arange(len(y_sticks)),
# #               labels=y_sticks)
#
# cbar = ax.figure.colorbar(im,
#                           ax = ax,
#                           shrink=0.5 )
# # ax.set_title("Matplotlib Heatmap with imshow", size=20)
# fig.tight_layout()

x=np.array(all_a_beams[i]).reshape((-1))


fig = go.Figure(go.Histogram2d(
    x=all_a_beams[i],
    y=all_b_beams[i]
))
fig.show()

In [ ]:
{'path':paths[i],
 'name':name,
 'type':types_dict[names[i]],
 'legend':[{'a_beams':legend1,'b_beams':legend2}],
 'density_curve_scatter':[{'a_beams':(x1.flatten(),y1.flatten()),'b_beams':(x2.flatten(),y2.flatten())}],
 'linear_approx_plot':[{'a_beams':(x_pred1.flatten(), y_pred1.flatten()),'b_beams':(x_pred2.flatten(), y_pred2.flatten())}],
 'linear_approx_data':[{'a_beams':{'k':k1,'b':b1,'angle':angle1,'score':score1},'b_beams':{'k':k2,'b':b2,'angle':angle2,'score':score2}}],
 'beams_length_series':[{'a_beams':all_a_beams,'b_beams':all_b_beams}],
 'pixel2meter':pixel,
 }

In [ ]:
data['data'][i]['beams_length_series'][0]['a_beams']

In [ ]:
matrices=[]
step=2
data = open(f'left_(1500,1500,1)_step_{step}_beams.json',encoding='utf-8')
data = json.load(data)

bin_max=30


ellipses_num=[]
ellispes_sum=[]

max_pairs=0

for  i in range(5):
    b_beams = np.array(data['data'][i]['beams_length_series'][0]['a_beams'])
    a_beams= np.array(data['data'][i]['beams_length_series'][0]['b_beams'])
    ellipses_num.append(len(a_beams))
    # plt.figure(figsize=(10,10))
    matrix,_, _, image = plt.hist2d(b_beams,a_beams,bins=range(0,bin_max+1))
    ellispes_sum.append(matrix.sum())
    matrix/=matrix.sum()
    # matrix=np.where(matrix>1e-3,matrix,0)
    # matrix=np.log(matrix)
    if matrix.max()>max_pairs:
        max_pairs=matrix.max()
    matrices.append(np.swapaxes(matrix,0,1))

In [ ]:
max_pairs

In [ ]:
cm = plt.cm.get_cmap('RdYlBu_r')

fig,axes=plt.subplots(1,5,figsize=(25,4))
fig.patch.set_facecolor('white')

font_size=20
plt.rcParams['font.size'] = f'{font_size}'

names=['Ultra_Co11 - средние зерна',
       'Ultra_Co15 - средне-мелкие зерна',
       'Ultra_Co25 - мелкие зерна',
       'Ultra_Co6_2 - крупные зерна',
       'Ultra_Co8 - средне-мелкие зерна']

ims=[]

normalizer = plt.Normalize(vmin=0, vmax=max_pairs)

for i,ax in enumerate(axes):
    im=ax.pcolormesh(matrices[i], cmap=plt.cm.RdYlBu_r)
    # im=ax.pcolormesh(matrices[i],norm=mpl.colors.LogNorm(), cmap=plt.cm.RdYlBu_r)
    # im=ax.pcolormesh(matrices[i],norm=normalizer, cmap=plt.cm.RdYlBu_r)
    ims.append(im)

coefs_list=[]
polynom_x=[]
polynom_y=[]


for i,ax in enumerate(axes):
    y=np.array(range(bin_max))
    x=[]
    for y_i in y:
        max_val=matrices[i][y_i].max()
        x.append(np.where(matrices[i][y_i]==max_val)[0][0])
    x=np.array(x)

    # (x_pred, y_pred), k, b, angle, score=polynom_regr_approx(x.reshape((-1,1)),y.reshape((-1,1)))
    coefs = np.polyfit(x.flatten(), y.flatten(), 2)

    # ax.plot(y_pred,x_pred,label=f'k={k}',color='green')
    x=np.array(range(bin_max))
    ax.plot(x,np.polyval(coefs, x),color='green')

    polynom_x.append(x)
    polynom_y.append(np.polyval(coefs, x))

    for j,coef in enumerate(coefs):
        coefs[j]=round(coef,3)

    coefs_list.append(coefs)

    # ax.legend([f'coefs={coefs}\nellipses num={ellipses_num[i]}\n norm value={ellispes_sum[i]}'])
    print(f'{names[i]}\ncoefs={coefs}\nellipses num={ellipses_num[i]}\n norm value={ellispes_sum[i]}')
    print('-----')
    # ax.scatter(x,y)

# big_im=copy.copy(ims[0])
#
# masked_data=big_im.__dict__['_A'].data
# for im in ims[1:]:
#     masked_data+= im.__dict__['_A'].data

# masked_data/=5
# new_bg_im=ma.masked_array(masked_data,mask=False)
# big_im.__dict__['_A']=new_bg_im


for i,ax in enumerate(axes):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(ims[i], cax=cax, orientation='vertical')
    # cbar = fig.colorbar(big_im, cax=cax, orientation='vertical')
    # cbar.set_label('log ( количество точек )',size=15)

    ax.set_xlabel('Малая полуось, мкм',fontsize=font_size)
    ax.set_ylabel('Большая полуось, мкм',fontsize=font_size)
    ax.set_title(names[i],fontsize=15)
    ax.set_xlim(0,bin_max)
    ax.set_ylim(0,bin_max)
    ax.grid(False)
    ax.set_facecolor("White")

# cax = plt.axes([0.8, 0.575, 0.035, 0.3])
# plt.colorbar(cax=cax)

# cbar.set_label('log ( количество точек )',size=15)
# ax1.set_ylabel('Большая полуось, мкм',fontsize=15)

# fig.colorbar(big_im, ax=axes.ravel().tolist())

fig.subplots_adjust(wspace=0.55)
plt.savefig(f'двумерное_распределение_полуосей_шаг_{step}.png',bbox_inches='tight')
plt.show()

names_new=['AB_Co11_medium',
       'AB_Co15_medium_small',
       'AB_Co25_small',
       'AB_Co6_large',
       'AB_Co_8_medium_small']

img_plot=io.imread(f'двумерное_распределение_полуосей_шаг_{step}.png')

for i in range(len(ims)):
    step_img=img_plot.shape[1]/len(ims)
    io.imsave(f'{names_new[i]}_step_{step}.png',img_plot[:,int(i*step_img):int((i+1)*step_img)])

In [ ]:
# plt.rcParams['axes.facecolor']='white'
fig=plt.figure(figsize=(10,8))
fig.patch.set_facecolor('white')

colors=['r','g','orange','b','y']

names_new=['AB_Co11_medium',
           'AB_Co15_medium_small',
           'AB_Co25_small',
           'AB_Co6_large',
           'AB_Co_8_medium_small']

legends=[]

for i in [2,0,3]:
    y=np.array(range(bin_max-1))
    x=[]
    for y_i in y:
        max_val=matrices[i][y_i].max()
        x.append(np.where(matrices[i][y_i]==max_val)[0][0])
    x=np.array(x)

    coefs=coefs_list[i]

    legends.append(f'{names[i]}\ncoefs={coefs}\nellipses num={ellipses_num[i]}')
    plt.plot(x, np.polyval(coefs, x),color=colors[i])
    # plt.scatter(y,x)

plt.legend(legends)

for i in range(5):
    y=np.array(range(bin_max-1))
    x=[]
    for y_i in y:
        max_val=matrices[i][y_i].max()
        x.append(np.where(matrices[i][y_i]==max_val)[0][0])
    x=np.array(x)

    plt.scatter(x,y,color=colors[i])

plt.grid(True)
plt.ylim(-2,bin_max)
plt.xlim(-2,bin_max)
plt.xlabel('Малая полуось')
plt.ylabel('Большая полуось')

plt.savefig('polynoms.png',bbox_inches='tight')
plt.show()

In [ ]:
# plt.rcParams['axes.facecolor']='white'
# fig=plt.figure(figsize=(4,4),facecolor='white')
fig=plt.figure(figsize=(10,10),facecolor='white')
# fig.patch.set_facecolor('white')

font_size=20
plt.rcParams.update({'font.size': font_size})

ax = plt.axes()
ax.set_facecolor("white")

colors = ['orange', 'red','blue','green', 'indigo']

legends=[]

for i in [2,0,3]:

    # legends.append(f'{names[i]}\ncoefs={coefs}\nellipses num={ellipses_num[i]}')
    legends.append(data['data'][i]['name']+' '+data['data'][i]['type'])

    x=polynom_x[i]
    y=polynom_y[i].astype(np.int32)

    y_clean=[]
    for y_i in y:
        if y_i<0:
            y_clean.append(0)
        if y_i<30 and y_i>=0:
            y_clean.append(y_i)

    x_clean=x[:len(y_clean)]
    y_clean=np.array(y_clean)

    # flipped=np.flip(matrices[i],0)
    # polynom_union_image=flipped[29-y_clean,x_clean]
    polynom_union_image=matrices[i][y_clean,x_clean]
    # polynom_union_image=np.log(polynom_union_image)

    plt.scatter(y_clean,polynom_union_image,color=colors[i])
    # plt.plot(y_clean,polynom_union_image,'o--',color=colors[i])

    # if i==0:
    #     for j,y_i in enumerate(y_clean):
    #         plt.text(y_i,polynom_union_image[j],f'y={y_i},x={j}')


plt.legend(legends)



plt.grid(True)
# plt.ylim(0,0.05)
# plt.xlim(0,30)
plt.xlabel('Большая полуось, мкм')
# plt.ylabel('цвет heatmap')
# plt.ylabel('log ( цвет heatmap )')
# plt.yticks(np.arange(0,0.035,0.01))
# plt.xticks(np.arange(0,25,5))
matplotlib.rc('xtick', labelsize=font_size)
matplotlib.rc('ytick', labelsize=font_size)


plt.grid(False)

# plt.savefig('polynoms_heatmp_0_30_y.png',bbox_inches='tight')
plt.show()

In [ ]:
np.flip(matrices[0],0)[29-4,3]

In [ ]:
matrices[0][11,7]

In [ ]:
y